# Imports and constants

In [1]:
#import statemets

import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from customScripts import utilities as util
from customScripts import features as feat
from customScripts import onset

import librosa

In [2]:
# parameters

#number of samples / second
sampling_rate = 96000

#length of frame in samples
frame_length = 2048 #48000

#number of samples used as offset for earch consecutive frame
hop_length = 1024 #24000

#number of mel frequency bins to use
bin_number = 40 #80

#set the desired number of frames / second here
ground_thruth_conversion_const = 100 #2

#input data
file_paths = ['music_data/shortName.flac']
grount_truth_paths = ['music_data/shortName.onsets']
# prediction_paths = ['predictions/Muppets-02-01-01.csv', 'predictions/Muppets-02-04-04.csv', 'predictions/Muppets-03-04-03.csv']
# file_lengths = [1547, 1548, 1539] #in seconds

# File loading and feature extraction

In [12]:
train_onsets_gt_names, train_beats_gt_names, train_bpm_gt_names, train_onsets_audio_names, train_beats_audio_names, train_bpm_audio_names = util.get_file_names('music_data/train')
#print(len(train_onsets_audio_names))
#print(len(train_beats_audio_names))
#print(len(train_bpm_audio_names))

test_onsets_gt_names, test_beats_gt_names, test_bpm_gt_names, test_onsets_audio_names, test_beats_audio_names, test_bpm_audio_names = util.get_file_names('music_data/test')
#print(len(test_onsets_audio_names))
#print(len(test_beats_audio_names))
#print(len(test_bpm_audio_names))

#print(train_onsets_audio_names[0])

321
177
107
71
0
0
D:\uni\2018WS\IAMA\exercise\repo\ex1\notebooks\music_data\train\SoundCheck2_60_Vocal_Tenor_opera.flac


In [9]:
#load audio
y_1 = util.load_audio(train_onsets_audio_names[0], sampling_rate)

In [10]:
#compute spectrograms
spectrogram_1 = feat.compute_spectrogram(y_1, frame_length, hop_length, bin_number)
spectrogram_1T = spectrogram_1.transpose()
print('frame number:',len(spectrogram_1T))
print('bin number:',len(spectrogram_1))

frame number: 5232
bin number: 40


In [ ]:
#load ground truth, pad it with zeroes and adjust to frame number

#episode 1
"""
gt_1 = util.load_ground_truth(grount_truth_paths[0], '/n')
gt_1 = util.compute_0_padded_gt(gt_1, file_lengths[0])
gt_1 = util.adjust_gt_to_frames(gt_1, ground_thruth_conversion_const)
# append zeros so that lenght equals the frames list length
for i in range(len(spectrogram_1T) - len(gt_1)):
    gt_1.append(0)
print('ground truth size:',len(gt_1))
"""

In [ ]:
#concatenate training data
"""
gt_train = gt_1 + gt_2

spectrogram_train = np.concatenate((spectrogram_1T, spectrogram_2T), axis=0)

spectrogram_predict = spectrogram_3T

#spectrogram_train[3195] == spectrogram_2T[100]
"""
#for i,g in enumerate(gt_train):
#    if g == 1:
#        print(i)

# Classic onset detection method

In [15]:
odf_1 = onset.compute_odf(spectrogram_1)
peaks_1 = onset.apply_threshold(odf_1, 1500)
#print(peaks_1)

for i, p in enumerate(peaks_1):
    if p > 0:
        print(i/100, '   ', p)

print(train_onsets_gt_names[0])
        
        
# maxima = o.pick_local_peaks(peaks)

# print(maxima)

0.16     4911.687614667323
0.17     3552.829881252663
0.63     1941.8000888872089
0.64     1692.6766333500432
0.65     1626.093914146778
0.85     2036.849908839988
1.37     1910.2390218664327
1.39     2018.7514144460258
1.68     4487.366100037915
1.7     1659.7639448022796
1.91     2142.0263151935633
3.4     2187.8340567633704
3.42     5084.729949639742
3.99     3707.5819156282364
4.0     2445.5957110759296
4.99     1544.060931938878
5.53     1587.0497328776692
7.01     3092.9744244063895
7.02     1555.8084514737336
7.3     2242.052599448835
7.52     2463.9116376886536
9.6     1675.896638076603
9.63     1847.9743087957797
9.66     1917.207417691683
10.15     1911.0561536804762
13.13     4899.612580836143
27.73     2216.2161098005868
28.3     1605.0368495252037
28.59     2935.736820641721
36.07     1754.7857735188145
36.08     1658.552512720222
36.59     2823.4605172942224
40.59     15242.250134518508
40.62     1872.800057328098
40.99     3656.443912174542
41.0     3207.202538269558
41.

# Classifier training and prediction for onsets

In [ ]:
# use a Multi-layer Perceptron classifier with default parameters
#clf = MLPClassifier()

In [ ]:
#clf.fit(spectrogram_train, gt_train)

In [ ]:
#prediction = clf.predict(spectrogram_predict)

In [ ]:
#np.savetxt(prediction_paths[2], prediction, delimiter="/n")

# Evaluation for onset detection

In [ ]:
"""
print('precision (micro average):',precision_score(gt_3, prediction, average='micro'))
print('precision (macro average):',precision_score(gt_3, prediction, average='macro'))
print('precision (weighted average):',precision_score(gt_3, prediction, average='weighted'))
"""

In [ ]:
"""
print('recall (micro average):',recall_score(gt_3, prediction, average='micro'))
print('recall (macro average):',recall_score(gt_3, prediction, average='macro'))
print('recall (weighted average):',recall_score(gt_3, prediction, average='weighted'))
"""

In [ ]:
"""
print('f1 (micro average):',f1_score(gt_3, prediction, average='micro'))
print('f1 (macro average):',f1_score(gt_3, prediction, average='macro'))
print('f1 (weighted average):',f1_score(gt_3, prediction, average='weighted'))
"""

In [ ]:
"""
# calculate false positive / true positive rate and area under curve
fpr, tpr, threshold = roc_curve(gt_3, prediction)
roc_auc = auc(fpr, tpr)

# plot ROC curve
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
"""